In [1]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential

from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam

from keras.applications import xception, inception_v3

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

Using TensorFlow backend.


In [2]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'

batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

### train

emsemble a few augmentation training data

In [3]:
base_model_x = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
model_x = add_preprocess(base_model_x, xception.preprocess_input)

In [4]:
base_model_i = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling='avg')
model_i = add_preprocess(base_model_i, inception_v3.preprocess_input)

In [5]:
batches = image.ImageDataGenerator().flow_from_directory(data_dir+'/train', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_val = image.ImageDataGenerator().flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_test = image.ImageDataGenerator().flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)
nb_batches_test = math.ceil(batches_test.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Found 10357 images belonging to 1 classes.


In [6]:
# bf_val_x = model_x.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_x', bf_val_x)
bf_val_x = np.load(data_dir+'/results/bf_val_x.npy')

# bf_x_test = model_x.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_x_test', bf_x_test)
bf_x_test = np.load(data_dir+'/results/bf_x_test.npy')

In [18]:
# bf_val_i = model_i.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_i', bf_val_i)
bf_val_i = np.load(data_dir+'/results/bf_val_i.npy')

# bf_i_test = model_i.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_i_test', bf_i_test)
bf_i_test = np.load(data_dir+'/results/bf_i_test.npy')

In [8]:
gen = image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=0.1,
                               shear_range=0.15, 
                               zoom_range=0.1, 
                               channel_shift_range=10., 
                               horizontal_flip=True)

In [38]:
preds = []
nb_runs = 10
for i in range(nb_runs):
    print("i:", i)
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    y = to_categorical(batches.classes)
    bf_x = model_x.predict_generator(batches, steps=nb_batches, verbose=1)
    
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    bf_i = model_i.predict_generator(batches, steps=nb_batches, verbose=1)
    
    lm = Sequential([Dense(120, activation='softmax', input_shape=(2048+2048,))])
    lm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    lm.fit(np.hstack([bf_x, bf_i]), y, epochs=15, batch_size=nb_batches, 
           validation_data=(np.hstack([bf_val_x, bf_val_i]), y_val))
    
    pred = lm.predict(np.hstack([bf_x_test, bf_i_test]), batch_size=batch_size, verbose=1)
    preds.append(pred)

i: 0
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 125s   
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 112s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 0s - loss: 2.0585 - acc: 0.6653 - val_loss: 0.7755 - val_acc: 0.8900
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 0.5829 - acc: 0.8909 - val_loss: 0.4558 - val_acc: 0.8950
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.3792 - acc: 0.9068 - val_loss: 0.3628 - val_acc: 0.8960
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.2868 - acc: 0.9284 - val_loss: 0.3211 - val_acc: 0.9055
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.2333 - acc: 0.9374 - val_loss: 0.3111 - val_acc: 0.8975
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.1939 - acc: 0.9462 - val_loss: 0.3134 - val_acc: 0.8990
Epoch 7

8222/8222 [==============================] - 0s - loss: 0.0807 - acc: 0.9814 - val_loss: 0.2813 - val_acc: 0.9090
Epoch 13/15
8222/8222 [==============================] - 0s - loss: 0.0690 - acc: 0.9860 - val_loss: 0.2894 - val_acc: 0.9015
Epoch 14/15
8222/8222 [==============================] - 0s - loss: 0.0615 - acc: 0.9872 - val_loss: 0.2810 - val_acc: 0.9095
Epoch 15/15
 9792/10357 [===========================>..] - ETA: 0si: 4
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 125s   
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 112s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 0s - loss: 2.0903 - acc: 0.6486 - val_loss: 0.7948 - val_acc: 0.8775
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 0.5960 - acc: 0.8862 - val_loss: 0.4514 - val_acc: 0.9015
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.3829 -

8222/8222 [==============================] - 0s - loss: 0.1647 - acc: 0.9550 - val_loss: 0.2954 - val_acc: 0.9085
Epoch 8/15
8222/8222 [==============================] - 0s - loss: 0.1413 - acc: 0.9638 - val_loss: 0.2879 - val_acc: 0.9050
Epoch 9/15
8222/8222 [==============================] - 0s - loss: 0.1193 - acc: 0.9703 - val_loss: 0.2859 - val_acc: 0.9045
Epoch 10/15
8222/8222 [==============================] - 0s - loss: 0.1067 - acc: 0.9732 - val_loss: 0.2833 - val_acc: 0.9060
Epoch 11/15
8222/8222 [==============================] - 0s - loss: 0.0926 - acc: 0.9771 - val_loss: 0.2837 - val_acc: 0.9075
Epoch 12/15
8222/8222 [==============================] - 0s - loss: 0.0799 - acc: 0.9827 - val_loss: 0.2805 - val_acc: 0.9080
Epoch 13/15
8222/8222 [==============================] - 0s - loss: 0.0720 - acc: 0.9855 - val_loss: 0.2777 - val_acc: 0.9135
Epoch 14/15
8222/8222 [==============================] - 0s - loss: 0.0630 - acc: 0.9861 - val_loss: 0.2933 - val_acc: 0.9045
Epoch 

In [39]:
pred_ensemble = np.stack(preds).mean(axis=0)

### predict

In [40]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [41]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred_ensemble]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [42]:
description = 'xception_inception_ensemble_%d_data_aug' % nb_runs
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [43]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [44]:
!kg submit $submission_file_name -m $description

0.26775
